In [3]:
import numpy as np
import numpy.linalg as LA
import matplotlib.pyplot as plt
import scipy.linalg as sla

In [4]:
# define lattice and generate hopping matrix with OBC

In [5]:
from pythtb import *
def Tmatrix(t,mu):

    # set model parameters
    mu=0.0
    t=-1.0
    lat=[[1.0,0.0],[0.0,1.0]]
    orb=[[0.0,0.0]]

    # create TB model
    my_model=tb_model(2,2,lat,orb)
    
    # set on-site energies
    my_model.set_onsite([-mu])
    # set hopping terms (one for each nearest neighbor)
    my_model.set_hop(t, 0, 0, [1, 0])
    my_model.set_hop(t, 0, 0, [0, 1])

    # create a 4x4 lattice
    Nx, Ny = 4, 4
    my_ham = my_model.cut_piece(Nx,0,glue_edgs=False)
    my_ham = my_ham.cut_piece(Ny,1,glue_edgs=False)
    # print Hamiltonian
    return my_ham._gen_ham()


In [120]:
def ExpM(phi,ExpT,Vdiag):
    # T = exp(-T*dtau)
    return np.diag(phi * Vdiag).dot(ExpT)

In [8]:
def SVD_stablizer_back(A,U,D,V):
    # B = U * D * V
    # B * A = U * D * V * B = U * U1 * D1 * V1 = U2 * D1 * V1
    U1, D, V = LA.svd(np.diag(D).dot(V).dot(A))
    return U.dot(U1), D, V
    
def SVD_stablizer_forward(A,U,D,V):
    # B = U * D * V
    # A * B = A * U * D * V = U1 * D1 * V1 * V = U1 * D1 * V2
    U, D, V1 = LA.svd(A.dot(U).dot(np.diag(D)))
    return U, D, V1.dot(V)

In [71]:
def GF(UL,DL,VL,UR,DR,VR):
    # B(t,0) = UL * DL * VL
    # B(beta,t) = VR * DR * UR Take care of it !!!
    U,D,V = LA.svd(LA.inv(UL.dot(UR))+np.diag(DR).dot(VR.dot(VL)).dot(np.diag(DL)))
    return LA.inv(V.dot(UL)).dot(np.diag(1/D)).dot(LA.inv(UR.dot(U)))

In [38]:
def Stable_Bmatrix(phi,ExpT,Vdiag):
    # use SVD algorithm to calculate the stable B matrix
    # ExpT: the matrix of exp(-dtau * T)
    # Vdiag: Ntau x L matrix with each column being - alpha 
    Ntau, L = phi.shape
    U = np.zeros((Ntau+1,L,L))
    D = np.zeros((Ntau+1,L))
    V = np.zeros((Ntau+1,L,L))

    U[0],D[0],V[0] = LA.svd(np.eye(L))
    for i in range(Ntau):
        U[i+1], D[i+1], V[i+1] = SVD_stablizer_forward(ExpM(phi[i], ExpT, Vdiag),U[i],D[i],V[i])
    return U,D,V

In [128]:
def Update_HS(GFup, GFdn, phi, alpha):
    Ident = np.eye(len(phi))
    for i in range(len(phi)):
        dup = np.exp(-2 * alpha * phi[i]) -1 
        ddn = np.exp(2 * alpha * phi[i]) -1
        R = (1 + dup * (1 - GFup[i,i]))*(1 + ddn * (1 - GFdn[i,i]))
        
        if  np.abs(R) > np.random.rand():
            phi[i] = -phi[i]
            GFup -= dup * np.outer(GFup[:,i], Ident[i]-GFup[i,:])/R
            GFdn -= ddn * np.outer(GFdn[:,i], Ident[i]-GFdn[i,:])/R
        
    return GFup, GFdn, phi

In [169]:
def Sweep_backward(GFup,GFdn,Uup,Dup,Vup,Udn,Ddn,Vdn,phi,ExpT,Vdiag,Nstable,alpha):
    # Input GFup, GFdn should be G(0,0) = G(\beta,\beta)
    # The first member of U, D, V should be the result of I = B(0,0) or B(\beta, \beta)
    Ntau = len(phi) 
    ind_revered = np.roll(np.arange(Ntau+1)[::-1], shift=1)

    for n in range(1,Ntau+1):
        i = ind_revered[n]
        i_1 = ind_revered[n-1]
        if np.mod(n,Nstable) == 0:
            # recompute equal time GF to avoid accumulation of numerical error
            GFup_recomputed = GF(Vup[i_1],Dup[i_1],Uup[i_1],Uup[i],Dup[i],Vup[i])
            GFdn_recomputed = GF(Vdn[i_1],Ddn[i_1],Udn[i_1],Udn[i],Dup[i],Vdn[i])

            # compare with advanced one to get the accumulated error
            GFup_error = np.max(np.abs(GFup_recomputed - GFup))
            GFdn_error = np.max(np.abs(GFdn_recomputed - GFdn))
            x = np.argmax(np.abs(GFup_recomputed - GFup))
            x1 = np.argmax(np.abs(GFdn_recomputed - GFdn))
            print("Error of Green function: ", (GFup_error + GFdn_error)/2)
            print(GFup.reshape(16*16)[x])
            print(GFdn.reshape(16*16)[x1])
            print(GFup_recomputed.reshape(16*16)[x])
            print(GFdn_recomputed.reshape(16*16)[x1])
            GFup = GFup_recomputed
            GFdn = GFdn_recomputed
        
        # Update H-S field and Green function
        GFup, GFdn, phi[i-1] = Update_HS(GFup, GFdn, phi[i-1], alpha)

        # Update SVD decompostion for B matrix
        Bup = ExpM(phi[i-1], ExpT,  Vdiag)
        Bdn = ExpM(phi[i-1], ExpT, -Vdiag)

        # Advance equal time GF using new H-S field
        GFup = LA.inv(Bup).dot(GFup).dot(Bup)
        GFdn = LA.inv(Bdn).dot(GFdn).dot(Bdn)

        # Repalce SVD decomposition of B matrix
        Uup[i], Dup[i], Vup[i] = SVD_stablizer_back(Bup, Uup[i_1], Dup[i_1], Vup[i_1])
        Udn[i], Ddn[i], Vdn[i] = SVD_stablizer_back(Bdn, Udn[i_1], Ddn[i_1], Vdn[i_1])

    return None


In [162]:
def Sweep_forward(GFup,GFdn,Uup,Dup,Vup,Udn,Ddn,Vdn,phi,ExpT,Vdiag,Nstable,alpha):
    # Input GFup, GFdn should be G(0,0) = G(\beta,\beta)
    # The first member of U, D, V should be the result of I = B(0,0) or B(\beta, \beta)

    # reverse the order of U, D, V, and shift B(0,0) to the first one, to make the sweep backward
    
    Ntau = len(phi) 
    for i in range(1,Ntau+1):
        if np.mod(i,Nstable) == 0:
            # recompute equal time GF to avoid accumulation of numerical error
            GFup_recomputed = GF(Vup[i],Dup[i],Uup[i],Uup[i-1],Dup[i-1],Vup[i-1])
            GFdn_recomputed = GF(Vdn[i],Ddn[i],Udn[i],Udn[i-1],Ddn[i-1],Vdn[i-1])

            # compare with advanced one to get the accumulated error
            GFup_error = np.max(np.abs(GFup_recomputed - GFup))
            GFdn_error = np.max(np.abs(GFdn_recomputed - GFdn))
            x = np.argmax(np.abs(GFup_recomputed - GFup))
            x1 = np.argmax(np.abs(GFdn_recomputed - GFdn))
            print("Error of Green function: ", (GFup_error + GFdn_error)/2)
            GFup = GFup_recomputed
            GFdn = GFdn_recomputed
        
        # Update H-S field and Green function
        GFup, GFdn, phi[i-1] = Update_HS(GFup, GFdn, phi[i-1],alpha)

        # Update SVD decompostion for B matrix
        Bup = ExpM(phi[i-1], ExpT,  Vdiag)
        Bdn = ExpM(phi[i-1], ExpT, -Vdiag)

        # Advance equal time GF using new H-S field
        GFup = Bup.dot(GFup).dot(LA.inv(Bup))
        GFdn = Bdn.dot(GFdn).dot(LA.inv(Bdn))

        # Repalce SVD decomposition of B matrix
        Uup[i], Dup[i], Vup[i] = SVD_stablizer_forward(Bup, Uup[i-1], Dup[i-1], Vup[i-1])
        Udn[i], Ddn[i], Vdn[i] = SVD_stablizer_forward(Bdn, Udn[i-1], Ddn[i-1], Vdn[i-1])

    return None

In [157]:
L = 4 * 4
Ntau = 20
Nstable = 5
dtau = 0.1
U = 4.0
t = -1.0
mu = 0
beta = dtau * Ntau
alpha = np.arccosh(np.exp(0.5*dtau*U))
T = Tmatrix(t,mu).real
ExpT = sla.expm(-dtau * T)
Vdiag = np.zeros(L)
Vdiag[:] = alpha

In [168]:
phi = np.random.choice([1, -1], size=(Ntau, L))
#part of spin up and spin down are not coupled but direct product
Uup,Dup,Vup = Stable_Bmatrix(phi,ExpT,Vdiag)
Udn,Ddn,Vdn = Stable_Bmatrix(phi,ExpT,-Vdiag)
GFup = GF(Vup[0],Dup[0],Uup[0],Uup[-1],Dup[-1],Vup[-1])
GFdn = GF(Vdn[0],Ddn[0],Udn[0],Udn[-1],Ddn[-1],Vdn[-1])

# Sweep
Sweep_backward(GFup,GFdn,Uup,Dup,Vup,Udn,Ddn,Vdn,phi,ExpT,Vdiag,Nstable,alpha)
Sweep_forward(GFup,GFdn,Uup,Dup,Vup,Udn,Ddn,Vdn,phi,ExpT,Vdiag,Nstable,alpha)

# Measurement 

Error of Green function:  0.008397195007179701
106
0.005380490810763295
0.011594467223477808
9.17553925314179e-05
8.88126273502854e-05
Error of Green function:  0.11062943762747873
255
1.0156804717785766
1.205131980267689
0.9997769032885555
0.9997766735027526
Error of Green function:  0.05009433744641449
118
0.03310322288768238
-0.0669973096382627
9.813529902775512e-06
9.795589678667733e-05
Error of Green function:  0.03598252654722712
221
1.0643291834174449
0.007919599976475447
1.0003297889530118
-4.6058653545756465e-05
Error of Green function:  0.017849869189160317
204


IndexError: index 204 is out of bounds for axis 0 with size 16